# Track the redun workflow run

Any redun run has inputs and outputs. LaminDB only tracks these inputs and outputs and the redun run execution ID.

Everything else is managed by redun internally.

In [ ]:
import lamindb as ln
import lamindb.schema as lns
import os
import json

ln.nb.header()

## Setting up a LaminDB run and querying input data

This is an artificial situation in which we pretend our input data comes from a notebook with ID `0ymQDuqM5Lwq`:

In [ ]:
input_dobjects = (
    ln.select(ln.DObject).join(lns.Run).join(lns.Notebook, id="0ymQDuqM5Lwq")
)

input_dobjects.df()

Let's create a LaminDB run record and link it agains the input files:

In [ ]:
pipeline = ln.select(lns.Pipeline, name="lamin-redun-fasta").one()  # load a pipeline
run = lns.Run(
    name="Test run", pipeline=pipeline
)  # create a run record that is linked against the pipeline
run.inputs = input_dobjects.all()  # link inputs to run
run = ln.add(run)  # add run to DB
run

## Call the redun CLI

In order to pass it to the redun CLI, export the `run.id` as an env variable:

In [ ]:
os.environ["LNDB_RUN_ID"] = run.id

Let us now call the workflow (tagging with the `run.id` is optional but ensures convenient query via `redun log` later):

In [ ]:
!redun run workflow.py main --run-id $LNDB_RUN_ID --tag run-id=$LNDB_RUN_ID  1> redun_stdout.txt 2>redun_stderr.txt

Inspect the output:

In [ ]:
!cat redun_stdout.txt

In [ ]:
!tail -1 redun_stderr.txt

In [ ]:
!redun log --exec --exec-tag run-id=$LNDB_RUN_ID --format json --no-pager > redun_exec.json

In [ ]:
redun_exec = json.load(open("redun_exec.json"))

redun_exec

## Track redun outputs and execution ID

In [ ]:
run = ln.select(lns.Run, id=run.id).one()
run.external_id = redun_exec["id"]
ln.add(run)

There is just a single output file to track, here:

In [ ]:
dobject = ln.DObject(data="data/results.tgz", source=run)
ln.add(dobject)

## View the LaminDB content

In [ ]:
ln.view()